<a href="https://colab.research.google.com/github/mrubash1/MatthewRubashkin_Insight_Homework/blob/master/caliper_cifar10_architecture_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To connect to a GPU runtime for faster training (highly recommended):
1. Go to Runtime > Change runtime type from topbar.
1. Select GPU from the Hardware accelerator dropdown list.

In [0]:
#@title Set Caliper credentials (Paste this notebook's URL here)
############################################################
##            DON'T EDIT CODE IN THIS CELL                ##
############################################################
NOTEBOOK_URL = "https://colab.research.google.com/drive/1NQaABGIfTd_VO6cAjivaCd8kyRtm8NaK#scrollTo=OIlDSqXu-QCi" #@param {type:"string"}
import urllib
parsed = urllib.parse.urlparse(NOTEBOOK_URL)
try:
  caliper_token = urllib.parse.parse_qs(parsed.query)['auth_token'][0]
  challenge_id = urllib.parse.parse_qs(parsed.query)['challenge_id'][0]
  phase_id = urllib.parse.parse_qs(parsed.query)['phase_id'][0]
except:
  print("Some error occured in parsing the pasted URL.")
############################################################
##            DON'T EDIT CODE IN THIS CELL                ##
############################################################

Some error occured in parsing the pasted URL.


In [0]:
#@title Install PyTorch

!pip install torch
!pip install torchvision

In [0]:
#@title Import dependencies

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import csv
from google.colab import files

In [0]:
#@title Set hyperparameters

num_classes = 20   # number of output classes discrete range [0,9]
num_epochs  = 7    # number of times the entire dataset is presented to the model
batch_size  = 64   # the size of input data took for one iteration
lr          = 1e-3 # size of step

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
#@title Download CIFAR-10 and set up dataloaders

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

train_data = dsets.CIFAR10(root='./data',
                           train=True,
                           transform=transform, download=True)

test_data = dsets.CIFAR10(root='./data', train=False,
                          transform=transform, download=True)

train_gen = torch.utils.data.DataLoader(dataset=train_data,
                                        batch_size=batch_size,
                                        shuffle=True, num_workers=2)

test_gen = torch.utils.data.DataLoader(dataset=test_data,
                                       batch_size=batch_size, 
                                       shuffle=False, num_workers=2)

In [0]:
#@title Download CIFAR-10 and set up dataloaders

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

train_data = dsets.CIFAR10(root='./data',
                           train=True,
                           transform=transform, download=True)

test_data = dsets.CIFAR10(root='./data', train=False,
                          transform=transform, download=True)

train_gen = torch.utils.data.DataLoader(dataset=train_data,
                                        batch_size=batch_size,
                                        shuffle=True, num_workers=2)

test_gen = torch.utils.data.DataLoader(dataset=test_data,
                                       batch_size=batch_size, 
                                       shuffle=False, num_workers=2)

0it [00:00, ?it/s]

170500096it [00:02, 72452604.77it/s]                               


Files already downloaded and verified


In [0]:
#@title Define the model

class Net(nn.Module):
  def __init__(self, num_classes):
    super(Net,self).__init__()
    self.conv1 = nn.Conv2d(3, 32, 3)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(32, 64, 3)
    self.conv3 = nn.Conv2d(64, 128, 3)
    self.fc1 = nn.Linear(128 * 2 * 2, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, num_classes)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(0.1)
  
  def forward(self,x):
    out = self.pool(self.relu(self.conv1(x)))
    out = self.pool(self.relu(self.conv2(out)))
    out = self.pool(self.relu(self.conv3(out)))
    out = out.view(-1, 128 * 2 * 2)
    out = self.dropout(self.relu(self.fc1(out)))
    out = self.dropout(self.relu(self.fc2(out)))
    out = self.fc3(out)
    return out
  
#@title Build the model

net = Net(num_classes)
net.to(device)

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=20, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1)
)

In [0]:
#@title Define loss function & optimizer

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [0]:
#@title Train model

net.train()
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_gen):
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = net(images)

    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_data)//batch_size,
                   loss.item()))

Epoch [1/7], Step [100/781], Loss: 1.9177
Epoch [1/7], Step [200/781], Loss: 1.6724
Epoch [1/7], Step [300/781], Loss: 1.7668
Epoch [1/7], Step [400/781], Loss: 1.3830
Epoch [1/7], Step [500/781], Loss: 1.4987
Epoch [1/7], Step [600/781], Loss: 1.4230
Epoch [1/7], Step [700/781], Loss: 1.2779
Epoch [2/7], Step [100/781], Loss: 1.3062
Epoch [2/7], Step [200/781], Loss: 1.3366
Epoch [2/7], Step [300/781], Loss: 1.1073
Epoch [2/7], Step [400/781], Loss: 1.0089
Epoch [2/7], Step [500/781], Loss: 0.8900
Epoch [2/7], Step [600/781], Loss: 0.9054
Epoch [2/7], Step [700/781], Loss: 1.0329
Epoch [3/7], Step [100/781], Loss: 0.9551
Epoch [3/7], Step [200/781], Loss: 0.8093
Epoch [3/7], Step [300/781], Loss: 0.9898
Epoch [3/7], Step [400/781], Loss: 0.9180
Epoch [3/7], Step [500/781], Loss: 0.8996
Epoch [3/7], Step [600/781], Loss: 0.8583
Epoch [3/7], Step [700/781], Loss: 1.0378
Epoch [4/7], Step [100/781], Loss: 0.8013
Epoch [4/7], Step [200/781], Loss: 0.9355
Epoch [4/7], Step [300/781], Loss:

In [0]:
#@title Run model on test set and save predictions

net.eval()
predictions = []

for images, _ in test_gen:
  images = images.to(device)

  output = net(images)
  _, predicted = torch.max(output, 1)
    
  img_id_start = len(predictions) + 1
  predictions += [{'image_id': img_id_start + x,
                   'label': predicted[x].item()} for x in range(len(predicted))]

with open('caliper_cifar10_test_predictions.csv', mode='w') as preds_file:
    writer = csv.writer(preds_file, delimiter=',')
    writer.writerow(['id','label'])
    for el in predictions:
        writer.writerow([el['image_id'], el['label']])
    
files.download('caliper_cifar10_test_predictions.csv')

In [0]:
#@title Upload predictions to Caliper
############################################################
##            DON'T EDIT CODE IN THIS CELL                ##
############################################################
import requests

if not (challenge_id and caliper_token):
    print("Could not locate caliper credentials. Please submit through Caliper website.")
else:
    submission_file = {'input_file': open('caliper_cifar10_test_predictions.csv','rb')}
    hostname = "https://caliper.ai"
    URL = "{0}/api/jobs/challenge/{1}/challenge_phase/{2}/token/{3}/submission/".format(hostname, challenge_id, phase_id, caliper_token)
    data = {"status": "submitting"}
    r = requests.post(URL, files=submission_file, data=data)
    result = r.json()

    if r.status_code == 201:
        print("Predictions submitted successfully. File URL: ", result.get('input_file'))
        print("Please go to {0}/web/challenges/{1}/submissions/phases/{2}/ to check the status of submission.".format(hostname, challenge_id, phase_id))
    else:
        print("Some error occured. Please try submitting the file through {0}/web/challenges/{1}/submission".format(hostname, challenge_id))
############################################################
##            DON'T EDIT CODE IN THIS CELL                ##
############################################################

NameError: ignored

If the above does not download a predictions CSV file to your machine (known issue on Firefox):

1. Go to View > Table of contents from topbar.
1. A left sidebar will show up. Open the Files tab.
1. Click Refresh.
1. Right-click and download caliper_mnist_test_predictions.csv.